# Applied Data Science Capstone
## 1) Introduction
##### The purpose of this project is to compare and contrast different aspects of places in New York City. We will specially focus on Manhattan and Bronx. The users will be able to determine which area is better for people to move in too.
# 2) Downloading and Prepping Data 
#### To be able to perform this data analysis, the data will be collected from NYU spatial data provider. The data is publicaly available at https://geo.nyu.edu/catalog/nyu_2451_34572

# 3) Methodology
##### For this report,  methods like,
##### I will be using folium for data visualization and choropleth maps for visualization also.
##### K-Means methods will also be used to cluster different aspects of the two boroughs in the New York City
##### After the data analysis, one will be able to decide which neighboorhood is the best option for them based on the common places available in the Bronx

#### 1) Installing libraries

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

In [2]:
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0           conda-forge
    geopy:          

In [3]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [4]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [5]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [6]:
#since all the necessary information is in the feature key
neighborhoods_data = newyork_data['features']

In [7]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

###### Viewing the dataframe that contains information on Borough, Neighborhood, Latitude, and Longitude

In [9]:
neighborhoods.head(10)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [10]:
#using the geopy library to get the latitude and logitude values of New York City
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [11]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


vincent-0.4.4        | 28 KB     | ##################################### | 1

### 2) Visualizing New York with its Borough and Neighnorhoods in the map

In [18]:
Bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
Bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [19]:
#geting the geographical coordinates of Bronx
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx are 40.8466508, -73.8785937.


In [20]:
# create map of Bronx using latitude and longitude values
map_Bronx = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Bronx_data['Latitude'], Bronx_data['Longitude'], Bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Bronx)  
    
map_Bronx

### 3) Exploring bronx using Foursquare Credentials

In [21]:
CLIENT_ID = '4K1XYU3JGNLJS4FHG4FSAV1FMC4NP4G5IKI2LQ5IHVSBSU3L' # your Foursquare ID
CLIENT_SECRET = 'PIGA0KV0JUOGQDQAX4Q0F52A3ACH2ZBD2WZZEWCWOD3FBFSR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4K1XYU3JGNLJS4FHG4FSAV1FMC4NP4G5IKI2LQ5IHVSBSU3L
CLIENT_SECRET:PIGA0KV0JUOGQDQAX4Q0F52A3ACH2ZBD2WZZEWCWOD3FBFSR


In [23]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=4K1XYU3JGNLJS4FHG4FSAV1FMC4NP4G5IKI2LQ5IHVSBSU3L&client_secret=PIGA0KV0JUOGQDQAX4Q0F52A3ACH2ZBD2WZZEWCWOD3FBFSR&v=20180605&ll=40.8466508,-73.8785937&radius=500&limit=100'

In [24]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5edbcca83907e7001b07f3ee'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bronx Park',
  'headerFullLocation': 'Bronx Park, Bronx',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': 40.8511508045, 'lng': -73.87265606159046},
   'sw': {'lat': 40.8421507955, 'lng': -73.88453133840953}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4abcfe4bf964a520fa8720e3',
       'name': 'JungleWorld',
       'location': {'address': 'The Bronx Zoo',
        'crossStreet': 'Asia Plaza',
        'lat': 40.84522652894162,
        'lng': -73.8771813890588,
        'labeledLatLngs': [{'label': 'display',
    

In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)


,name,categories,lat,lng
0,JungleWorld,Zoo,40.845227,-73.877181
1,African Lions,Zoo Exhibit,40.847058,-73.878024
2,Congo Gorilla Forest,Zoo,40.847774,-73.881604
3,Grizzly Corner,Zoo Exhibit,40.849023,-73.877739
4,Himalayan Highlands,Scenic Lookout,40.848404,-73.876649
5,Giraffe House,Zoo Exhibit,40.847875,-73.880127
6,Wild Asia Monorail,Zoo,40.845285,-73.875788
7,Bug Carousel,Theme Park,40.849461,-73.879659
8,World of Reptiles,Zoo Exhibit,40.849287,-73.878856
9,Zoo Store,Souvenir Shop,40.850120,-73.880204


# Note: From this we can see that Zoo is the most popular place to visit in the Bronx

In [28]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


In [29]:
Bronx_data.loc[0, 'Neighborhood']
neighborhood_latitude = Bronx_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Bronx_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Bronx_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [58]:
# type your answer here

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
results = requests.get(url).json()
results
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

25 venues were returned by Foursquare.


In [32]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [59]:


Bronx_venues = getNearbyVenues(names=Bronx_data['Neighborhood'],
                                   latitudes=Bronx_data['Latitude'],
                                   longitudes=Bronx_data['Longitude']
                                  )



Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [34]:
print(Bronx_venues.shape)
Bronx_venues.head()

(1224, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


In [35]:
print('There are {} uniques categories.'.format(len(Bronx_venues['Venue Category'].unique())))

There are 167 uniques categories.


In [36]:
# one hot encoding
Bronx_onehot = pd.get_dummies(Bronx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Bronx_onehot['Neighborhood'] = Bronx_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Bronx_onehot.columns[-1]] + list(Bronx_onehot.columns[:-1])
Bronx_onehot = Bronx_onehot[fixed_columns]

Bronx_onehot.head()

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Bike Trail,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Music Store,Music Venue,Nightclub,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Wakefield,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [37]:
Bronx_onehot.shape

(1224, 168)

In [38]:
Bronx_grouped = Bronx_onehot.groupby('Neighborhood').mean().reset_index()
Bronx_grouped

,Neighborhood,Accessories Store,African Restaurant,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Bar,Bike Trail,Boat or Ferry,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Cafeteria,Café,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Donut Shop,Eastern European Restaurant,Electronics Store,Eye Doctor,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hotel,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Lake,Latin American Restaurant,Laundromat,Lawyer,Liquor Store,Lounge,Market,Martial Arts Dojo,Mattress Store,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Music Store,Music Venue,Nightclub,Outdoors & Recreation,Outlet Store,Paella Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Platform,Playground,Plaza,Pool,Print Shop,Pub,Recording Studio,Recreation Center,Rental Car Location,Restaurant,River,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Social Club,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tattoo Parlor,Tennis Court,Tennis Stadium,Thai Restaurant,Thrift / Vintage Store,Track,Trail,Train Station,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waste Facility,Wings Joint,Women's Store
0,Allerton,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.000000,0.068966,0.000000,0.034483,0.000000,0.034483,0.000000,0.000000,0.000000,0.034483,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.034483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.137931,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.034483,0.000000,0.000000,0.000000,0.000000,0.068966,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Baychester,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.00

In [39]:
num_top_venues = 5

for hood in Bronx_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Bronx_grouped[Bronx_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
                venue  freq
0         Pizza Place  0.14
1       Deli / Bodega  0.07
2  Chinese Restaurant  0.07
3         Supermarket  0.07
4  Spanish Restaurant  0.03


----Baychester----
                  venue  freq
0            Donut Shop  0.11
1           Men's Store  0.06
2           Supermarket  0.06
3  Fast Food Restaurant  0.06
4             Pet Store  0.06


----Bedford Park----
                venue  freq
0  Chinese Restaurant  0.11
1               Diner  0.11
2         Pizza Place  0.08
3  Mexican Restaurant  0.08
4       Deli / Bodega  0.08


----Belmont----
                venue  freq
0  Italian Restaurant  0.18
1         Pizza Place  0.08
2       Deli / Bodega  0.07
3              Bakery  0.05
4                Bank  0.03


----Bronxdale----
                           venue  freq
0             Spanish Restaurant  0.08
1  Paper / Office Supplies Store  0.08
2             Italian Restaurant  0.08
3                    Pizza Place  0.08
4                 Brea

### 4) Below is the listed Most common places in the Neighborhood of the Bronx

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Bronx_grouped['Neighborhood']

for ind in np.arange(Bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Deli / Bodega,Chinese Restaurant,Supermarket,Bus Station,Spanish Restaurant,Martial Arts Dojo,Pharmacy,Food,Fast Food Restaurant
1,Baychester,Donut Shop,Pizza Place,Bank,Mattress Store,Fast Food Restaurant,Electronics Store,Spanish Restaurant,Sporting Goods Shop,Sandwich Place,Supermarket
2,Bedford Park,Chinese Restaurant,Diner,Pizza Place,Deli / Bodega,Mexican Restaurant,Pharmacy,Sandwich Place,Supermarket,Baseball Field,Park
3,Belmont,Italian Restaurant,Pizza Place,Deli / Bodega,Bakery,Donut Shop,Dessert Shop,Bank,Grocery Store,Sandwich Place,Bar
4,Bronxdale,Pizza Place,Performing Arts Venue,Breakfast Spot,Spanish Restaurant,Eastern European Restaurant,Supermarket,Paper / Office Supplies Store,Chinese Restaurant,Bank,Italian Restaurant


## Note: From this we can see that Italian Cuisine is more common in the Neighboorhood of bronx

### 5) Clustering the Bronx Neighborhoods

In [48]:
# set number of clusters
kclusters = 5

Bronx_grouped_clustering = Bronx_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bronx_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [51]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Bronx_merged = Bronx_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Bronx_merged = Bronx_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Bronx_merged.head(11) # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bronx,Wakefield,40.894705,-73.847201,0,Pharmacy,Ice Cream Shop,Laundromat,Deli / Bodega,Dessert Shop,Sandwich Place,Donut Shop,Gas Station,Food,Grocery Store
1,Bronx,Co-op City,40.874294,-73.829939,1,Bus Station,Baseball Field,Pizza Place,Park,Mattress Store,Fried Chicken Joint,Salon / Barbershop,Restaurant,Bagel Shop,Discount Store
2,Bronx,Eastchester,40.887556,-73.827806,1,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Convenience Store,Food & Drink Shop,Cosmetics Shop,Chinese Restaurant,Donut Shop,Seafood Restaurant
3,Bronx,Fieldston,40.895437,-73.905643,4,Plaza,River,Bus Station,Distillery,Farmers Market,Eye Doctor,Electronics Store,Eastern European Restaurant,Donut Shop,Doctor's Office
4,Bronx,Riverdale,40.890834,-73.912585,1,Park,Playground,Bank,Plaza,Food Truck,Bus Station,Gym,Baseball Field,Eye Doctor,Electronics Store
5,Bronx,Kingsbridge,40.881687,-73.902818,1,Pizza Place,Bar,Deli / Bodega,Sandwich Place,Latin American Restaurant,Supermarket,Mexican Restaurant,Bakery,Fast Food Restaurant,Pharmacy
6,Bronx,Woodlawn,40.898273,-73.867315,1,Deli / Bodega,Pub,Pizza Place,Playground,Bakery,Cosmetics Shop,Park,Rental Car Location,Bus Stop,Donut Shop
7,Bronx,Norwood,40.877224,-73.879391,1,Pizza Place,Park,Bank,Pharmacy,American Restaurant,Chinese Restaurant,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Bus Station
8,Bronx,Williamsbridge,40.881039,-73.857446,1,Playground,Soup Place,Nightclub,Bar,Caribbean Restaurant,Doctor's Office,Farmers Market,Eye Doctor,Electronics Store,Eastern European Restaurant
9,Bronx,Baychester,40.866858,-73.835798,0,Donut Shop,Pizza Place,Bank,Mattress Store,Fast Food Restaurant,Electronics Store,Spanish Restaurant,Sporting Goods Shop,Sandwich Place,Supermarket


### Note:- Below is the clustered map of the neighboorhood of bronx regarding the common places they have

In [52]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bronx_merged['Latitude'], Bronx_merged['Longitude'], Bronx_merged['Neighborhood'], Bronx_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  From the above diagram:
### The neighborhood clustered in color dots are similar to each other based on the Bronx


### Looking at every cluster Individually in a dataframe

In [53]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 0, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Wakefield,Pharmacy,Ice Cream Shop,Laundromat,Deli / Bodega,Dessert Shop,Sandwich Place,Donut Shop,Gas Station,Food,Grocery Store
9,Baychester,Donut Shop,Pizza Place,Bank,Mattress Store,Fast Food Restaurant,Electronics Store,Spanish Restaurant,Sporting Goods Shop,Sandwich Place,Supermarket
15,Fordham,Shoe Store,Mobile Phone Shop,Donut Shop,Bank,Pizza Place,Fast Food Restaurant,Spanish Restaurant,Pharmacy,Supplement Shop,Bakery
20,Mott Haven,Pizza Place,Gym,Spanish Restaurant,Donut Shop,Peruvian Restaurant,Burger Joint,Fish & Chips Shop,Grocery Store,Mobile Phone Shop,Pharmacy
21,Port Morris,Spanish Restaurant,Furniture / Home Store,Peruvian Restaurant,Distillery,Donut Shop,Music Venue,Brewery,Metro Station,Bookstore,Storage Facility
22,Longwood,Deli / Bodega,Fast Food Restaurant,Diner,Sandwich Place,Grocery Store,Latin American Restaurant,Donut Shop,Women's Store,Doctor's Office,Farmers Market
24,Morrisania,Bus Station,Discount Store,Donut Shop,Grocery Store,Fast Food Restaurant,Fish Market,Bowling Alley,Pharmacy,Sandwich Place,Seafood Restaurant
30,Westchester Square,Fast Food Restaurant,Metro Station,Pharmacy,Sandwich Place,Pizza Place,Donut Shop,Baseball Field,Building,Mexican Restaurant,Latin American Restaurant
36,Pelham Bay,Italian Restaurant,Bank,Diner,Sandwich Place,Gym / Fitness Center,Fast Food Restaurant,Donut Shop,Convenience Store,Lawyer,Chinese Restaurant
37,Schuylerville,Pizza Place,Pharmacy,Diner,Mexican Restaurant,Bar,Bank,Hookah Bar,Sandwich Place,Donut Shop,Fast Food Restaurant


In [54]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 1, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bus Station,Baseball Field,Pizza Place,Park,Mattress Store,Fried Chicken Joint,Salon / Barbershop,Restaurant,Bagel Shop,Discount Store
2,Eastchester,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Convenience Store,Food & Drink Shop,Cosmetics Shop,Chinese Restaurant,Donut Shop,Seafood Restaurant
4,Riverdale,Park,Playground,Bank,Plaza,Food Truck,Bus Station,Gym,Baseball Field,Eye Doctor,Electronics Store
5,Kingsbridge,Pizza Place,Bar,Deli / Bodega,Sandwich Place,Latin American Restaurant,Supermarket,Mexican Restaurant,Bakery,Fast Food Restaurant,Pharmacy
6,Woodlawn,Deli / Bodega,Pub,Pizza Place,Playground,Bakery,Cosmetics Shop,Park,Rental Car Location,Bus Stop,Donut Shop
7,Norwood,Pizza Place,Park,Bank,Pharmacy,American Restaurant,Chinese Restaurant,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Bus Station
8,Williamsbridge,Playground,Soup Place,Nightclub,Bar,Caribbean Restaurant,Doctor's Office,Farmers Market,Eye Doctor,Electronics Store,Eastern European Restaurant
10,Pelham Parkway,Pizza Place,Deli / Bodega,Bus Station,Italian Restaurant,Sushi Restaurant,Frozen Yogurt Shop,Mexican Restaurant,Smoke Shop,Donut Shop,Sandwich Place
11,City Island,Harbor / Marina,Grocery Store,Seafood Restaurant,Thrift / Vintage Store,History Museum,Baseball Field,Pizza Place,Park,Diner,Music Venue
12,Bedford Park,Chinese Restaurant,Diner,Pizza Place,Deli / Bodega,Mexican Restaurant,Pharmacy,Sandwich Place,Supermarket,Baseball Field,Park


In [55]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 1, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Co-op City,Bus Station,Baseball Field,Pizza Place,Park,Mattress Store,Fried Chicken Joint,Salon / Barbershop,Restaurant,Bagel Shop,Discount Store
2,Eastchester,Caribbean Restaurant,Deli / Bodega,Diner,Bus Station,Convenience Store,Food & Drink Shop,Cosmetics Shop,Chinese Restaurant,Donut Shop,Seafood Restaurant
4,Riverdale,Park,Playground,Bank,Plaza,Food Truck,Bus Station,Gym,Baseball Field,Eye Doctor,Electronics Store
5,Kingsbridge,Pizza Place,Bar,Deli / Bodega,Sandwich Place,Latin American Restaurant,Supermarket,Mexican Restaurant,Bakery,Fast Food Restaurant,Pharmacy
6,Woodlawn,Deli / Bodega,Pub,Pizza Place,Playground,Bakery,Cosmetics Shop,Park,Rental Car Location,Bus Stop,Donut Shop
7,Norwood,Pizza Place,Park,Bank,Pharmacy,American Restaurant,Chinese Restaurant,Fast Food Restaurant,Sandwich Place,Mexican Restaurant,Bus Station
8,Williamsbridge,Playground,Soup Place,Nightclub,Bar,Caribbean Restaurant,Doctor's Office,Farmers Market,Eye Doctor,Electronics Store,Eastern European Restaurant
10,Pelham Parkway,Pizza Place,Deli / Bodega,Bus Station,Italian Restaurant,Sushi Restaurant,Frozen Yogurt Shop,Mexican Restaurant,Smoke Shop,Donut Shop,Sandwich Place
11,City Island,Harbor / Marina,Grocery Store,Seafood Restaurant,Thrift / Vintage Store,History Museum,Baseball Field,Pizza Place,Park,Diner,Music Venue
12,Bedford Park,Chinese Restaurant,Diner,Pizza Place,Deli / Bodega,Mexican Restaurant,Pharmacy,Sandwich Place,Supermarket,Baseball Field,Park


In [56]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 3, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Country Club,Sandwich Place,Playground,Athletics & Sports,Women's Store,Dive Bar,Fast Food Restaurant,Farmers Market,Eye Doctor,Electronics Store,Eastern European Restaurant


In [57]:
Bronx_merged.loc[Bronx_merged['Cluster Labels'] == 4, Bronx_merged.columns[[1] + list(range(5, Bronx_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Fieldston,Plaza,River,Bus Station,Distillery,Farmers Market,Eye Doctor,Electronics Store,Eastern European Restaurant,Donut Shop,Doctor's Office


### From the above Data exploration using Four Square Website we can say that Italian Cuissine is more popular in Bronx
### We can also see that some of the neighboorhood in the Bronx doesnot have Chinese quisine as a common place. Opening a Chinese restautrent in the neighboorhood might be a good investment.